In [ ]:
import csv
import pandas as pd
import numpy as np
import string
import nltk
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
%matplotlib inline
pd.set_option('display.max_colwidth',100)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def load_data():
  data = pd.read_csv('drive/MyDrive/SKRIPSI/DatasetVaksinFix.csv',encoding='ISO-8859-1')

  return data


In [ ]:
tweet_df = load_data()
tweet_df.head(1000)

,Tanggal,Tweets,Label
0,11/13/2021 3:56,Vaksin pertama berasa digigit semut sekampung\nVaksin kedua berasa digigit semut megalodon sekam...,Negatif
1,11/11/2021 3:00,"Tanpa disadari, warga Republik Indonesia hidup berdampingan dengan covid-19... Vaksin Pulihkan R...",Positif
2,11/12/2021 5:10,"Puluhan ASN Terjaring Razia Vaksin di Kota Subulussalam, Aceh. https://t.co/3dOfuCA10e",Negatif
3,11/11/2021 1:03,iya deh yg bangga masih pure bloood. kita yang vaksin mah udah ternoda penuh toxin,Negatif
4,11/9/2021 11:44,Iya deh nanti insyaallah kalau ada program vaksin lagi di kantor desa,Positif
...,...,...,...
673,11/15/2021 14:55,Jae aku juga abis di vaksin tangan aku sakit pegel gitu,Negatif
674,11/15/2021 2:35,"Vaksin untuk keluarga sehat, Maju Bersama Jokowi https://t.co/JJMibq2mrH",Positif
675,11/18/2021 9:23,Baru aja balik Vaksin kedua,Netral
676,11/19/2021 13:21,"disuntik vaksin biar kuat fisiknya, yang lemah hatinya",Negatif


In [ ]:
tweet_df["Label"].value_counts()

Negatif    312
Positif    216
Netral     150
Name: Label, dtype: int64

In [ ]:
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678 entries, 0 to 677
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Tanggal  678 non-null    object
 1   Tweets   678 non-null    object
 2   Label    678 non-null    object
dtypes: object(3)
memory usage: 16.0+ KB


In [ ]:
df = pd.DataFrame(tweet_df[['Tweets']])

In [ ]:
######################################
#TAHAPAN MEMULAI Cleansing
####################################
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def remove_punct(Tweets):
  Tweets = "".join([char for char in Tweets if char not in string.punctuation])
  Tweets = re.sub('[0-9]+', '', Tweets)
  Tweets = re.sub(r"http\S+", "", Tweets)
  Tweets = re.sub(r"www.\S+", "", Tweets)

  return Tweets

df['Tweets'] = df['Tweets'].apply(lambda x: remove_punct(x))
df.head(700)

,Tweets
0,Vaksin pertama berasa digigit semut sekampung\nVaksin kedua berasa digigit semut megalodon sekam...
1,Tanpa disadari warga Republik Indonesia hidup berdampingan dengan covid Vaksin Pulihkan RI
2,Puluhan ASN Terjaring Razia Vaksin di Kota Subulussalam Aceh
3,iya deh yg bangga masih pure bloood kita yang vaksin mah udah ternoda penuh toxin
4,Iya deh nanti insyaallah kalau ada program vaksin lagi di kantor desa
...,...
673,Jae aku juga abis di vaksin tangan aku sakit pegel gitu
674,Vaksin untuk keluarga sehat Maju Bersama Jokowi
675,Baru aja balik Vaksin kedua
676,disuntik vaksin biar kuat fisiknya yang lemah hatinya


In [ ]:
df['Tweets'].to_csv("drive/MyDrive/SKRIPSI/Data-hasil-cleansing.csv",encoding='utf-8',index=False)

In [ ]:
######################################
#TAHAPAN MEMULAI CASEFOLDING
####################################
data1 = pd.read_csv('drive/MyDrive/SKRIPSI/Data-hasil-cleansing.csv',encoding='ISO-8859-1')
data1

,Tweets
0,Vaksin pertama berasa digigit semut sekampung\nVaksin kedua berasa digigit semut megalodon sekam...
1,Tanpa disadari warga Republik Indonesia hidup berdampingan dengan covid Vaksin Pulihkan RI
2,Puluhan ASN Terjaring Razia Vaksin di Kota Subulussalam Aceh
3,iya deh yg bangga masih pure bloood kita yang vaksin mah udah ternoda penuh toxin
4,Iya deh nanti insyaallah kalau ada program vaksin lagi di kantor desa
...,...
673,Jae aku juga abis di vaksin tangan aku sakit pegel gitu
674,Vaksin untuk keluarga sehat Maju Bersama Jokowi
675,Baru aja balik Vaksin kedua
676,disuntik vaksin biar kuat fisiknya yang lemah hatinya


In [ ]:
datacf = pd.DataFrame(data1['Tweets'].str.lower())
datacf

,Tweets
0,vaksin pertama berasa digigit semut sekampung\nvaksin kedua berasa digigit semut megalodon sekam...
1,tanpa disadari warga republik indonesia hidup berdampingan dengan covid vaksin pulihkan ri
2,puluhan asn terjaring razia vaksin di kota subulussalam aceh
3,iya deh yg bangga masih pure bloood kita yang vaksin mah udah ternoda penuh toxin
4,iya deh nanti insyaallah kalau ada program vaksin lagi di kantor desa
...,...
673,jae aku juga abis di vaksin tangan aku sakit pegel gitu
674,vaksin untuk keluarga sehat maju bersama jokowi
675,baru aja balik vaksin kedua
676,disuntik vaksin biar kuat fisiknya yang lemah hatinya


In [ ]:
#Simpan data hasil CF ke dalam drive
datacf.to_csv("drive/MyDrive/SKRIPSI/Data-hasil-casefolding.csv",encoding='utf-8',index=False)

In [ ]:
######################################
#TAHAPAN MEMULAI TOKENIZING
####################################
data2 = pd.read_csv('drive/MyDrive/SKRIPSI/Data-hasil-casefolding.csv',encoding='ISO-8859-1')
data2

,Tweets
0,vaksin pertama berasa digigit semut sekampung\nvaksin kedua berasa digigit semut megalodon sekam...
1,tanpa disadari warga republik indonesia hidup berdampingan dengan covid vaksin pulihkan ri
2,puluhan asn terjaring razia vaksin di kota subulussalam aceh
3,iya deh yg bangga masih pure bloood kita yang vaksin mah udah ternoda penuh toxin
4,iya deh nanti insyaallah kalau ada program vaksin lagi di kantor desa
...,...
673,jae aku juga abis di vaksin tangan aku sakit pegel gitu
674,vaksin untuk keluarga sehat maju bersama jokowi
675,baru aja balik vaksin kedua
676,disuntik vaksin biar kuat fisiknya yang lemah hatinya


In [ ]:
def tokenization(Tweets):
  Tweets = re.split('\W+', Tweets)

  return Tweets

df['Tweets'] = df['Tweets'].apply(lambda x: tokenization(x.lower()))
df.head(700)

,Tweets
0,"[vaksin, pertama, berasa, digigit, semut, sekampung, vaksin, kedua, berasa, digigit, semut, mega..."
1,"[tanpa, disadari, warga, republik, indonesia, hidup, berdampingan, dengan, covid, vaksin, pulihk..."
2,"[puluhan, asn, terjaring, razia, vaksin, di, kota, subulussalam, aceh, ]"
3,"[iya, deh, yg, bangga, masih, pure, bloood, kita, yang, vaksin, mah, udah, ternoda, penuh, toxin, ]"
4,"[iya, deh, nanti, insyaallah, kalau, ada, program, vaksin, lagi, di, kantor, desa]"
...,...
673,"[jae, aku, juga, abis, di, vaksin, tangan, aku, sakit, pegel, gitu]"
674,"[vaksin, untuk, keluarga, sehat, maju, bersama, jokowi, ]"
675,"[baru, aja, balik, vaksin, kedua]"
676,"[disuntik, vaksin, biar, kuat, fisiknya, yang, lemah, hatinya]"


In [ ]:
#Simpan data hasil tokenization ke dalam drive
df['Tweets'].to_csv("drive/MyDrive/SKRIPSI/Data-hasil-tokenization.csv",encoding='utf-8',index=False)

In [ ]:
######################################
#TAHAPAN MEMULAI NORMALISASI
####################################
normalizad_word = pd.read_excel("drive/MyDrive/SKRIPSI/normalisasi.xlsx")

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

df['Tweets'] = df['Tweets'].apply(normalized_term)

df['Tweets'].head(700)

0      [vaksin, pertama, berasa, digigit, semut, sekampung, vaksin, kedua, berasa, digigit, semut, mega...
1      [tanpa, disadari, warga, republik, indonesia, hidup, berdampingan, dengan, covid, vaksin, pulihk...
2                                 [puluhan, asn, terjaring, razia, vaksin, di, kota, subulussalam, aceh, ]
3      [iya, deh, yang, bangga, masih, pure, bloood, kita, yang, vaksin, mah, sudah, ternoda, penuh, to...
4                       [iya, deh, nanti, insyaallah, kalau, ada, program, vaksin, lagi, di, kantor, desa]
                                                      ...                                                 
673                                 [jae, aku, juga, habis, di, vaksin, tangan, aku, sakit, pegel, begitu]
674                                              [vaksin, untuk, keluarga, sehat, maju, bersama, jokowi, ]
675                                                                     [baru, saja, balik, vaksin, kedua]
676                                  

In [ ]:
#Simpan data hasil Normalisasi ke dalam drive
df['Tweets'].to_csv("drive/MyDrive/SKRIPSI/Data-hasil-normalisasi.csv",encoding='utf-8',index=False)

In [ ]:
######################################
#TAHAPAN MEMULAI STOPWORD
####################################
def stopwords(Tweets):
  reg = re.compile(r"\n")
  return reg.split(Tweets)

file = open("drive/MyDrive/SKRIPSI/stopword.txt","r");
st = file.read()
stopwords = stopwords(st)
stopwords

['ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'anda',
 'andalah',
 'antar',
 'antara',
 'antaranya',
 'apa',
 'apaan',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'artinya',
 'asal',
 'asalkan',
 'atas',
 'atau',
 'ataukah',
 'ataupun',
 'awal',
 'awalnya',
 'bagai',
 'bagaikan',
 'bagaimana',
 'bagaimanakah',
 'bagaimanapun',
 'bagi',
 'bagian',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'baik',
 'bakal',
 'bakalan',
 'balik',
 'banyak',
 'bapak',
 'baru',
 'bawah',
 'beberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'bekerja',
 'belakang',
 'belakangan',
 'belum',
 'belumlah',
 'benar',
 'benarkah',
 'benarlah',
 'berada',
 'berakhir',
 'berakhirlah',
 'berakhirnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'berarti',
 'berawal',
 'berbagai',
 'berdatangan',
 'beri',
 'berikan',
 'berikut'

In [ ]:
data3 = pd.read_csv('drive/MyDrive/SKRIPSI/Data-hasil-normalisasi.csv',encoding='ISO-8859-1')
data3

,Tweets
0,"['vaksin', 'pertama', 'berasa', 'digigit', 'semut', 'sekampung', 'vaksin', 'kedua', 'berasa', 'd..."
1,"['tanpa', 'disadari', 'warga', 'republik', 'indonesia', 'hidup', 'berdampingan', 'dengan', 'covi..."
2,"['puluhan', 'asn', 'terjaring', 'razia', 'vaksin', 'di', 'kota', 'subulussalam', 'aceh', '']"
3,"['iya', 'deh', 'yang', 'bangga', 'masih', 'pure', 'bloood', 'kita', 'yang', 'vaksin', 'mah', 'su..."
4,"['iya', 'deh', 'nanti', 'insyaallah', 'kalau', 'ada', 'program', 'vaksin', 'lagi', 'di', 'kantor..."
...,...
673,"['jae', 'aku', 'juga', 'habis', 'di', 'vaksin', 'tangan', 'aku', 'sakit', 'pegel', 'begitu']"
674,"['vaksin', 'untuk', 'keluarga', 'sehat', 'maju', 'bersama', 'jokowi', '']"
675,"['baru', 'saja', 'balik', 'vaksin', 'kedua']"
676,"['disuntik', 'vaksin', 'biar', 'kuat', 'fisiknya', 'yang', 'lemah', 'hatinya']"


In [ ]:
def remove_stopwords(Tweets):
  Tweets = [word for word in Tweets if word not in stopwords]
  return Tweets

df['Tweets'] = df['Tweets'].apply(lambda x: remove_stopwords(x))
df.head(700)

,Tweets
0,"[vaksin, berasa, digigit, semut, sekampung, vaksin, berasa, digigit, semut, megalodon, sekampung, ]"
1,"[disadari, warga, republik, indonesia, hidup, berdampingan, covid, vaksin, pulihkan, ri, ]"
2,"[puluhan, asn, terjaring, razia, vaksin, kota, subulussalam, aceh, ]"
3,"[iya, deh, bangga, pure, bloood, vaksin, mah, ternoda, penuh, toxin, ]"
4,"[iya, deh, insyaallah, program, vaksin, kantor, desa]"
...,...
673,"[jae, habis, vaksin, tangan, sakit, pegel]"
674,"[vaksin, keluarga, sehat, maju, jokowi, ]"
675,[vaksin]
676,"[disuntik, vaksin, biar, kuat, fisiknya, lemah, hatinya]"


In [ ]:
#Simpan data hasil tokenization ke dalam drive
df['Tweets'].to_csv("drive/MyDrive/SKRIPSI/Data-hasil-stopword.csv",encoding='utf-8',index=False)

In [ ]:
#Hasil stopword
data4 = pd.read_csv('drive/MyDrive/SKRIPSI/Data-hasil-stopword.csv',encoding='ISO-8859-1')
data4

,Tweets
0,"['vaksin', 'berasa', 'digigit', 'semut', 'sekampung', 'vaksin', 'berasa', 'digigit', 'semut', 'm..."
1,"['disadari', 'warga', 'republik', 'indonesia', 'hidup', 'berdampingan', 'covid', 'vaksin', 'puli..."
2,"['puluhan', 'asn', 'terjaring', 'razia', 'vaksin', 'kota', 'subulussalam', 'aceh', '']"
3,"['iya', 'deh', 'bangga', 'pure', 'bloood', 'vaksin', 'mah', 'ternoda', 'penuh', 'toxin', '']"
4,"['iya', 'deh', 'insyaallah', 'program', 'vaksin', 'kantor', 'desa']"
...,...
673,"['jae', 'habis', 'vaksin', 'tangan', 'sakit', 'pegel']"
674,"['vaksin', 'keluarga', 'sehat', 'maju', 'jokowi', '']"
675,['vaksin']
676,"['disuntik', 'vaksin', 'biar', 'kuat', 'fisiknya', 'lemah', 'hatinya']"
